In [1]:
import numpy as np
import tensorflow as tf

/Users/sw/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/sw/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
is_training = tf.constant(True)

initializer = tf.contrib.layers.xavier_initializer()
act_fn = tf.nn.relu

def conv2d_layer(input_layer, out_channel, k_h=3, k_w=3, d_h=1, d_w=1, padding='SAME', name='conv_layer'):
    with tf.variable_scope(name):
        in_channel = input_layer.get_shape()[-1]
        w = tf.get_variable('w', [k_h, k_w, in_channel, out_channel], initializer=initializer)
        conv = tf.nn.conv2d(input_layer, w, strides=[1, d_h, d_w, 1], padding=padding, name='conv')
        bias = tf.get_variable('b', [out_channel], initializer=tf.constant_initializer(0))
        conv_b = tf.nn.bias_add(conv, bias)
        
        return conv_b
    

def batch_norm(inputs, train_phase=True, name="batch_norm"):
    train_bn = tf.contrib.layers.batch_norm(inputs=inputs, decay=0.9, updates_collections=None, zero_debias_moving_mean=True,
                                           is_training=True, reuse=None, scale=True, epsilon=1e-5, trainable=True, scope=name)
    test_bn = tf.contrib.layers.batch_norm(inputs=inputs, decay=0.9, updates_collections=None, zero_debias_moving_mean=True, 
                                           is_training=False, reuse=True, scale=True, epsilon=1e-5, trainable=True, scope=name)
    
    return tf.cond(train_phase, lambda: train_bn, lambda: test_bn)

def conv2d_block(input_layer, out_channel, k_h=3, k_w=3, d_h=1, d_w=1, padding='SAME', activation_fn=act_fn, name='conv_block'):
    with tf.variable_scope(name):
        conv = conv2d_layer(input_layer, out_channel, k_h, k_w, d_h, d_w, padding=padding)
        bn = batch_norm(conv, train_phase=is_training, name='batch_norm')
        act = activation_fn(bn, 'act')
        return act
    
    
def conv2d_trans(input_layer, k_h=2, k_w=2, d_h=2, d_w=2, activation_fn=act_fn, name="conv2d_trans"):
    with tf.variable_scope(name):
        deconv_shape = tf.stack([tf.shape(input_layer)[0], input_layer.get_shape()[1] * 2,
                                 input_layer.get_shape()[2] * 2, input_layer.get_shape()[-1]])
        w = tf.get_variable('w', [k_h, k_w, input_layer.get_shape()[-1], input_layer.get_shape()[-1]],
                           initializer=initializer)
        conv = tf.nn.conv2d_transpose(input_layer, w, output_shape=deconv_shape, strides=[1, d_h, d_w, 1], padding='SAME')
        bias = tf.get_variable('biases', [input_layer.get_shape()[-1]], initializer=tf.constant_initializer(0))
        conv_b = tf.nn.bias_add(conv, bias)
        
        bn = batch_norm(conv_b, train_phase=is_training, name='batch_norm')
        act = activation_fn(bn, 'act')
        
        return act

In [3]:
kernel_num = 64
output_dim = 2

def unet(inputs):
    with tf.variable_scope('encoder'):
        layer_1_1 = conv2d_block(inputs, kernel_num, padding='SAME', name='conv_1_1')
        layer_1_2 = conv2d_block(layer_1_1, kernel_num, padding='SAME', name='conv_1_2')
        layer_pool_1 = tf.nn.max_pool(layer_1_2, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding='SAME', name='max_1')
        
        layer_2_1 = conv2d_block(layer_pool_1, kernel_num * 2, padding='SAME', name='conv_2_1')
        layer_2_2 = conv2d_block(layer_2_1, kernel_num * 2, padding='SAME', name='conv_2_2')
        layer_pool_2 = tf.nn.max_pool(layer_2_2, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding='SAME', name='max_2')
        
        layer_3_1 = conv2d_block(layer_pool_2, kernel_num * 4, padding='SAME', name='conv_3_1')
        layer_3_2 = conv2d_block(layer_3_1, kernel_num * 4, padding='SAME', name='conv_3_2')
        layer_pool_3 = tf.nn.max_pool(layer_3_2, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding='SAME', name='max_3')
        
        layer_4_1 = conv2d_block(layer_pool_3, kernel_num * 8, padding='SAME', name='conv_4_1')
        layer_4_2 = conv2d_block(layer_4_1, kernel_num * 8, padding='SAME', name='conv_4_2')
        layer_pool_4 = tf.nn.max_pool(layer_4_2, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding='SAME', name='max_4')
        
    with tf.variable_scope('bridge'):
        bridge_1 = conv2d_block(layer_pool_4, kernel_num * 16, padding='SAME', name='bridge_1')
        bridge_2 = conv2d_block(bridge_1, kernel_num * 16, padding='SAME', name='bridge_2')
        
    with tf.variable_scope('decoder'):
        decoder_1_1 = conv2d_trans(bridge_2, name='deconv_1_1')
        decoder_1_2 = conv2d_block(tf.concat([layer_4_2, decoder_1_1], axis=-1), kernel_num * 8, padding='SAME', name='deconv_1_2')
        decoder_1_3 = conv2d_block(decoder_1_2, kernel_num * 8, padding='SAME', name='deconv_1_3')
        
        decoder_2_1 = conv2d_trans(decoder_1_3, name='deconv_2_1')
        decoder_2_2 = conv2d_block(tf.concat([layer_3_2, decoder_2_1], axis=-1), kernel_num * 8, padding='SAME', name='deconv_2_2')
        decoder_2_3 = conv2d_block(decoder_2_2, kernel_num * 4, padding='SAME', name='deconv_2_3')
        
        decoder_3_1 = conv2d_trans(decoder_2_3, name='deconv_3_1')
        decoder_3_2 = conv2d_block(tf.concat([layer_2_2, decoder_3_1], axis=-1), kernel_num * 8, padding='SAME', name='deconv_3_2')
        decoder_3_3 = conv2d_block(decoder_3_2, kernel_num * 2, padding='SAME', name='deconv_3_3')
        
        decoder_4_1 = conv2d_trans(decoder_3_3, name='deconv_4_1')
        decoder_4_2 = conv2d_block(tf.concat([layer_1_2, decoder_4_1], axis=-1), kernel_num * 8, padding='SAME', name='deconv_4_2')
        decoder_4_3 = conv2d_block(decoder_4_2, kernel_num, padding='SAME', name='deconv_4_3')
        
    with tf.variable_scope('output'):
        output = conv2d_layer(decoder_4_3, output_dim, k_h=1, k_w=1, d_h=1, d_w=1, padding='SAME', name='output')
        
    return output


        


In [4]:
input_layer = tf.placeholder(dtype=tf.float32, shape=(None, 128, 128, 3), name='input_image')
pred=unet(input_layer)
pred

<tf.Tensor 'output/output/BiasAdd:0' shape=(?, 128, 128, 2) dtype=float32>